In [8]:
# 引入套件
import pandas as pd
import numpy as np

In [ ]:
import openpyxl
df = pd.read_excel("../文章原始資料/0926_new.xlsx")

In [17]:
df = pd.read_csv("../文章原始資料/Scitechvista.csv")

In [18]:
df

,title,tag,content,url
0,OK繃、便利貼超方便，都是感壓膠的功勞！,橡膠(5) 便利貼(3) 黏著力(4),除了前述的Dahlquist準則外，玻璃轉變溫度必須在適當範圍。研究人員在1986年提出「感...,https://scitechvista.nat.gov.tw/Article/C00000...
1,共生質療法,益生菌(18) 敗血症(2) 安慰劑(8) 新生兒(2) 雙盲實驗(1),"實驗採雙盲模式進行，實驗組與對照組各有2,278名新生兒。孩子出生後4天內開始餵食共生質或安...",https://scitechvista.nat.gov.tw/Article/C00000...
2,複合材料的創新與利用,複合材料(6),複合材料可依其內含物的幾何形狀分為：顆粒型、纖維型、層狀複合材料，當然也可以是不同內含物的組...,https://scitechvista.nat.gov.tw/Article/C00000...
3,能自動分解的塑膠–可降解塑膠,熱固塑膠(2),塑膠是一種大家熟悉的材料，顧名思義，它是在一定溫度和壓力下，可以塑製成形的一種合成高分子材料...,https://scitechvista.nat.gov.tw/Article/C00000...
4,仿生蝙蝠（四）：回聲和壓力感應的導盲帽,回聲定位(2) 蝙蝠(16),責任編輯：歐陽盛芝／國立臺灣博物館 （以上新聞編譯自2015年9月11日發行之Interna...,https://scitechvista.nat.gov.tw/Article/C00000...
...,...,...,...,...
5755,走調的粉彩青春派對（二）：塵爆自救要訣及傷患分類,植皮(1),八仙樂園塵爆事故是921大地震迄今臺灣受傷人數最多的災難，傷者多為二度以上臉部及肢體燒傷。事...,https://scitechvista.nat.gov.tw/Article/C00000...
5756,團結力量大,腦瘤(4),腦瘤病人中，4成以上是神經膠瘤，而大部分神經膠瘤是惡性的。這種腦瘤禁得住放射線療法，又能侵入...,https://scitechvista.nat.gov.tw/Article/C00000...
5757,失智病患照顧者的「保護因子」,失智症(20),近年來失智症（dementia）已成為席捲全球的疾病，根據國際衛生組織（WHO）及國際失智症...,https://scitechvista.nat.gov.tw/Article/C00000...
5758,拒絕無腦賭博！怎麼用賠率算出贏的機率？,機率論(2),每逢重大的運動競賽，常會聽到賠率一詞。網路上也經常有人在問賠率的意義。賠率與機率關係密切，利...,https://scitechvista.nat.gov.tw/Article/C00000...


In [6]:
# Import modules
from sentence_transformers import SentenceTransformer, util
# Select model by transformer
# about model: https://huggingface.co/sentence-transformers/paraphrase-multilingual-mpnet-base-v2
sbert_model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')
print('sbert_model loaded')


sbert_model loaded


In [9]:
def embedding_sentence(sentence):
    try:
        return {
            "state" : True,
            "value" : sbert_model.encode(sentence).tolist(), # return a vector list
        }
    except Exception as exp:
        return {
            "state" : False,
            "value" : np.zeros(768)
        }


In [19]:
# 假设你的DataFrame为df
df['vector_title'] = df['title'].apply(lambda x: embedding_sentence(x)['value'])

In [20]:
df['vector_tag'] = df['tag'].apply(lambda x: embedding_sentence(x)['value'])

In [21]:
df['vector_content'] = df['content'].apply(lambda x: embedding_sentence(x)['value'])

In [31]:
df.to_json("../backup/pansci_vec.json")

In [32]:
import numpy as np

def convert_to_np_array(lst):
    # 檢查 lst 是否為 None 或為具有值的列表
    if lst is None:
        return np.zeros(768)
    elif isinstance(lst, list):
        return np.array(lst)
    else:
        return np.zeros(768)


#####################
# Request Variables
# -------------------
# set : list of dict
#   - weight : float
#   - array : np.array
# len_array : int (length of array)
#####################
import numpy as np

def calc_array_mean(input_set, len_array):
    # Define weight that needs to sum
    sum_weight = 0
    # Define sum_array to sum all arrays
    sum_array = np.zeros(len_array)
    for item in input_set:
        item['array'] = np.array(item['array'])  # Convert to NumPy array
        # Check if the array is empty
        if item['array'].sum() == 0:
            # Don't calculate this weight
            sum_weight += 0
        else:
            # Calculate this weight
            sum_weight += item['weight']
            # Add to sum_array
            sum_array += item['array'] * item['weight']

    return sum_array / sum_weight


In [33]:
# 定义计算平均值的函数
def calc_array_mean(row):
    weights = [3, 1, 1]  # 根据你的要求定义权重
    vectors = [np.array(row['vector_title']), np.array(row['vector_tag']), np.array(row['vector_content'])]
    
    sum_weight = sum(weights)
    weighted_sum = sum(w * v for w, v in zip(weights, vectors))
    
    return weighted_sum / sum_weight

# 计算新列"vector"的值
df['vector'] = df.apply(calc_array_mean, axis=1)

In [30]:
df.to_json("pansci_vec.json")